In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import json

In [42]:
with open('Data/incomeTax.json') as data_file:    
    data = json.load(data_file)

# Testing
print data[0]["data"]["New Brunswick"][0]["rate"]
print data[0]["data"]["New Brunswick"][0]["upper"]

9.68
39305


In [73]:
def getAvgRate(location, income, data):
    
    def govtAvg(govt):
        # Fetch the tax bracket data
        brackets = data[govt]
        untaxed_income = income # This will keep track of the part of the income that has not been taxed yet
        tax_paid = 0.0 # A running total of the amount of tax paid
        lower = 0 # The lower value of tax bracket
        n = 0 # Keeps track of which tax bracket we're in
        while untaxed_income > 0:
            # Fetch the current bracket
            bracket = brackets[n]
            # This checks to see if we're on the final bracket
            try:
                upper = int(bracket["upper"]) # Upper value of tax bracket
                rate = float(bracket["rate"])/100.0 # Tax rate of bracket
            except:
                upper = "max" # There is no upper value for the final bracket
                rate = float(bracket["rate"])/100.0

            if upper == "max" or income < upper:
                # Does the calculation for the final bracket the person falls into
                tax_paid = tax_paid + untaxed_income*rate
                untaxed_income = 0
            else:
                # Gets the tax paid on the brackets a person is totally above
                tax_paid = tax_paid + (upper - lower)*rate
                untaxed_income = untaxed_income - (upper - lower)
                lower = upper
                n = n + 1
        # This is the average rate the person ends up paying
        return round(tax_paid / income * 100.0, 2)
    
    avg_fed_tax = govtAvg("Federal")
    avg_prov_tax = govtAvg(location)
    
    avg_tax_total = round(avg_prov_tax + avg_fed_tax, 2)

    return {"avg_fed": avg_fed_tax, "avg_prov": avg_prov_tax, "avg_total": avg_tax_total}

In [77]:
# For testing the above funciton
year = data[0]["year"]
year_data = data[0]["data"]
tax = getAvgRate("Nova Scotia", 1000000, year_data)
print tax

{'avg_total': 48.04, 'avg_prov': 20.11, 'avg_fed': 27.93}


In [72]:
for year_record in data:
    year = year_record["year"]
    print year

2014


In [71]:
data

[{u'data': {u'Federal': [{u'rate': u'15', u'upper': u'43953'},
    {u'rate': u'22', u'upper': u'87907'},
    {u'rate': u'26', u'upper': u'136270'},
    {u'rate': u'29', u'upper': u'Above'}],
   u'New Brunswick': [{u'rate': u'9.68', u'upper': u'39305'},
    {u'rate': u'14.82', u'upper': u'78609'},
    {u'rate': u'16.52', u'upper': u'127802'},
    {u'rate': u'17.84', u'upper': u'Above'}],
   u'Newfoundland and Labrador': [{u'rate': u'7.7', u'upper': u'34254'},
    {u'rate': u'12.5', u'upper': u'68508'},
    {u'rate': u'13.3', u'upper': u'Above'}],
   u'Nova Scotia': [{u'rate': u'8.79', u'upper': u'29590'},
    {u'rate': u'14.95', u'upper': u'59180'},
    {u'rate': u'16.67', u'upper': u'93000'},
    {u'rate': u'17.5', u'upper': u'150000'},
    {u'rate': u'21', u'upper': u'Above'}],
   u'Ontario': [{u'rate': u'5.05', u'upper': u'40120'},
    {u'rate': u'9.15', u'upper': u'80242'},
    {u'rate': u'11.16', u'upper': u'150000'},
    {u'rate': u'12.16', u'upper': u'220000'},
    {u'rate': u'13